#  ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA. Use Ironhack's database to load the pokemon data (db: pokemon, table: pokemon_stats). 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the data:
pokemon = pd.read_csv('pokemon.csv')
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [62]:
# Your code here
# Your code here
def combo(value):
    type1 = value['Type 1'];
    type2 = value['Type 2']
    if str(type2) == 'nan':
        return type1
    else:
        return type1 +'-'+ type2
    
pokemon['unique_types'] = pokemon.apply(combo, axis=1)

unique_types = []
for x in pokemon['unique_types']:
    if x not in unique_types:
        unique_types.append(x)

In [63]:
len(unique_types)

154

In [74]:
pokemon_totals=[]
for x in unique_types:
    total=0
    for i in range(0,len(pokemon)):
        if pokemon['unique_types'][i]==x:
            total+=pokemon['Total'][i]
    pokemon_totals.append(total)    
pokemon_totals

[6211,
 11542,
 3309,
 634,
 24411,
 4925,
 5873,
 4175,
 8927,
 24910,
 5689,
 11166,
 5138,
 1010,
 6074,
 1650,
 1235,
 2304,
 7899,
 1670,
 17653,
 1280,
 2280,
 2405,
 1325,
 1535,
 1815,
 845,
 1365,
 13318,
 2382,
 760,
 2828,
 2963,
 2506,
 2098,
 910,
 2688,
 4547,
 3850,
 1916,
 790,
 950,
 2698,
 610,
 670,
 3685,
 2110,
 4339,
 4362,
 2470,
 4296,
 1055,
 3568,
 2140,
 1120,
 1305,
 1100,
 940,
 410,
 1230,
 1715,
 465,
 1430,
 1220,
 1300,
 600,
 630,
 3450,
 1040,
 1295,
 269,
 1570,
 690,
 236,
 860,
 1330,
 1290,
 2470,
 1200,
 1325,
 860,
 590,
 880,
 800,
 850,
 850,
 5793,
 1230,
 3420,
 2600,
 770,
 525,
 530,
 410,
 1370,
 846,
 1180,
 1308,
 485,
 2610,
 1150,
 330,
 790,
 1422,
 480,
 440,
 520,
 520,
 520,
 520,
 680,
 600,
 1360,
 600,
 508,
 760,
 1162,
 1074,
 836,
 810,
 815,
 791,
 794,
 1165,
 471,
 786,
 830,
 1320,
 910,
 580,
 580,
 1160,
 680,
 680,
 2060,
 423,
 876,
 495,
 1813,
 770,
 320,
 494,
 770,
 883,
 883,
 500,
 431,
 1800,
 4099,
 780,
 600

In [76]:
new = pd.DataFrame({"Types":unique_types, "Totals":pokemon_totals})

In [81]:
q = new.set_index('Types').T
q

In [105]:
new['Types_count'] = new.groupby('Types').cumcount() ##is the new index 
new
new_pivot = new.pivot(index='Types_count', columns='Types', values='Totals')
new_pivot
new_pivot.columns = ['Type_'+str(x) for x in new_pivot.columns.values]
new_pivot

,Type_Bug,Type_Bug-Electric,Type_Bug-Fighting,Type_Bug-Fire,Type_Bug-Flying,Type_Bug-Ghost,Type_Bug-Grass,Type_Bug-Ground,Type_Bug-Poison,Type_Bug-Rock,...,Type_Water-Fighting,Type_Water-Flying,Type_Water-Ghost,Type_Water-Grass,Type_Water-Ground,Type_Water-Ice,Type_Water-Poison,Type_Water-Psychic,Type_Water-Rock,Type_Water-Steel
Types_count,,,,,,,,,,,,,,,,,,,,,
0,4925,791,1100,910,5873,236,2304,690,4175,1305,...,1670,2828,815,1040,4339,1535,1280,2405,1715,530


In [107]:
list_cheating = []
for c in new_pivot.columns:
    a = 'new_pivot' + '["' + c + '"],'
    list_cheating.append(a)
list_cheating

['new_pivot["Type_Bug"],',
 'new_pivot["Type_Bug-Electric"],',
 'new_pivot["Type_Bug-Fighting"],',
 'new_pivot["Type_Bug-Fire"],',
 'new_pivot["Type_Bug-Flying"],',
 'new_pivot["Type_Bug-Ghost"],',
 'new_pivot["Type_Bug-Grass"],',
 'new_pivot["Type_Bug-Ground"],',
 'new_pivot["Type_Bug-Poison"],',
 'new_pivot["Type_Bug-Rock"],',
 'new_pivot["Type_Bug-Steel"],',
 'new_pivot["Type_Bug-Water"],',
 'new_pivot["Type_Dark"],',
 'new_pivot["Type_Dark-Dragon"],',
 'new_pivot["Type_Dark-Fighting"],',
 'new_pivot["Type_Dark-Fire"],',
 'new_pivot["Type_Dark-Flying"],',
 'new_pivot["Type_Dark-Ghost"],',
 'new_pivot["Type_Dark-Ice"],',
 'new_pivot["Type_Dark-Psychic"],',
 'new_pivot["Type_Dark-Steel"],',
 'new_pivot["Type_Dragon"],',
 'new_pivot["Type_Dragon-Electric"],',
 'new_pivot["Type_Dragon-Fairy"],',
 'new_pivot["Type_Dragon-Fire"],',
 'new_pivot["Type_Dragon-Flying"],',
 'new_pivot["Type_Dragon-Ground"],',
 'new_pivot["Type_Dragon-Ice"],',
 'new_pivot["Type_Dragon-Psychic"],',
 'new_pivot["

In [108]:
k = ''.join(list_cheating)
k

'new_pivot["Type_Bug"],new_pivot["Type_Bug-Electric"],new_pivot["Type_Bug-Fighting"],new_pivot["Type_Bug-Fire"],new_pivot["Type_Bug-Flying"],new_pivot["Type_Bug-Ghost"],new_pivot["Type_Bug-Grass"],new_pivot["Type_Bug-Ground"],new_pivot["Type_Bug-Poison"],new_pivot["Type_Bug-Rock"],new_pivot["Type_Bug-Steel"],new_pivot["Type_Bug-Water"],new_pivot["Type_Dark"],new_pivot["Type_Dark-Dragon"],new_pivot["Type_Dark-Fighting"],new_pivot["Type_Dark-Fire"],new_pivot["Type_Dark-Flying"],new_pivot["Type_Dark-Ghost"],new_pivot["Type_Dark-Ice"],new_pivot["Type_Dark-Psychic"],new_pivot["Type_Dark-Steel"],new_pivot["Type_Dragon"],new_pivot["Type_Dragon-Electric"],new_pivot["Type_Dragon-Fairy"],new_pivot["Type_Dragon-Fire"],new_pivot["Type_Dragon-Flying"],new_pivot["Type_Dragon-Ground"],new_pivot["Type_Dragon-Ice"],new_pivot["Type_Dragon-Psychic"],new_pivot["Type_Electric"],new_pivot["Type_Electric-Dragon"],new_pivot["Type_Electric-Fairy"],new_pivot["Type_Electric-Fire"],new_pivot["Type_Electric-Flying

In [109]:
st.f_oneway(new_pivot["Type_Bug"],new_pivot["Type_Bug-Electric"],new_pivot["Type_Bug-Fighting"],new_pivot["Type_Bug-Fire"],new_pivot["Type_Bug-Flying"],new_pivot["Type_Bug-Ghost"],new_pivot["Type_Bug-Grass"],new_pivot["Type_Bug-Ground"],new_pivot["Type_Bug-Poison"],new_pivot["Type_Bug-Rock"],new_pivot["Type_Bug-Steel"],new_pivot["Type_Bug-Water"],new_pivot["Type_Dark"],new_pivot["Type_Dark-Dragon"],new_pivot["Type_Dark-Fighting"],new_pivot["Type_Dark-Fire"],new_pivot["Type_Dark-Flying"],new_pivot["Type_Dark-Ghost"],new_pivot["Type_Dark-Ice"],new_pivot["Type_Dark-Psychic"],new_pivot["Type_Dark-Steel"],new_pivot["Type_Dragon"],new_pivot["Type_Dragon-Electric"],new_pivot["Type_Dragon-Fairy"],new_pivot["Type_Dragon-Fire"],new_pivot["Type_Dragon-Flying"],new_pivot["Type_Dragon-Ground"],new_pivot["Type_Dragon-Ice"],new_pivot["Type_Dragon-Psychic"],new_pivot["Type_Electric"],new_pivot["Type_Electric-Dragon"],new_pivot["Type_Electric-Fairy"],new_pivot["Type_Electric-Fire"],new_pivot["Type_Electric-Flying"],new_pivot["Type_Electric-Ghost"],new_pivot["Type_Electric-Grass"],new_pivot["Type_Electric-Ice"],new_pivot["Type_Electric-Normal"],new_pivot["Type_Electric-Steel"],new_pivot["Type_Electric-Water"],new_pivot["Type_Fairy"],new_pivot["Type_Fairy-Flying"],new_pivot["Type_Fighting"],new_pivot["Type_Fighting-Dark"],new_pivot["Type_Fighting-Flying"],new_pivot["Type_Fighting-Psychic"],new_pivot["Type_Fighting-Steel"],new_pivot["Type_Fire"],new_pivot["Type_Fire-Dragon"],new_pivot["Type_Fire-Fighting"],new_pivot["Type_Fire-Flying"],new_pivot["Type_Fire-Ground"],new_pivot["Type_Fire-Normal"],new_pivot["Type_Fire-Psychic"],new_pivot["Type_Fire-Rock"],new_pivot["Type_Fire-Steel"],new_pivot["Type_Fire-Water"],new_pivot["Type_Flying"],new_pivot["Type_Flying-Dragon"],new_pivot["Type_Ghost"],new_pivot["Type_Ghost-Dark"],new_pivot["Type_Ghost-Dragon"],new_pivot["Type_Ghost-Fire"],new_pivot["Type_Ghost-Flying"],new_pivot["Type_Ghost-Grass"],new_pivot["Type_Ghost-Poison"],new_pivot["Type_Grass"],new_pivot["Type_Grass-Dark"],new_pivot["Type_Grass-Dragon"],new_pivot["Type_Grass-Fairy"],new_pivot["Type_Grass-Fighting"],new_pivot["Type_Grass-Flying"],new_pivot["Type_Grass-Ground"],new_pivot["Type_Grass-Ice"],new_pivot["Type_Grass-Poison"],new_pivot["Type_Grass-Psychic"],new_pivot["Type_Grass-Steel"],new_pivot["Type_Ground"],new_pivot["Type_Ground-Dark"],new_pivot["Type_Ground-Dragon"],new_pivot["Type_Ground-Electric"],new_pivot["Type_Ground-Fire"],new_pivot["Type_Ground-Flying"],new_pivot["Type_Ground-Ghost"],new_pivot["Type_Ground-Psychic"],new_pivot["Type_Ground-Rock"],new_pivot["Type_Ground-Steel"],new_pivot["Type_Ice"],new_pivot["Type_Ice-Flying"],new_pivot["Type_Ice-Ghost"],new_pivot["Type_Ice-Ground"],new_pivot["Type_Ice-Psychic"],new_pivot["Type_Ice-Water"],new_pivot["Type_Normal"],new_pivot["Type_Normal-Fairy"],new_pivot["Type_Normal-Fighting"],new_pivot["Type_Normal-Flying"],new_pivot["Type_Normal-Grass"],new_pivot["Type_Normal-Ground"],new_pivot["Type_Normal-Psychic"],new_pivot["Type_Normal-Water"],new_pivot["Type_Poison"],new_pivot["Type_Poison-Bug"],new_pivot["Type_Poison-Dark"],new_pivot["Type_Poison-Dragon"],new_pivot["Type_Poison-Fighting"],new_pivot["Type_Poison-Flying"],new_pivot["Type_Poison-Ground"],new_pivot["Type_Poison-Water"],new_pivot["Type_Psychic"],new_pivot["Type_Psychic-Dark"],new_pivot["Type_Psychic-Fairy"],new_pivot["Type_Psychic-Fighting"],new_pivot["Type_Psychic-Fire"],new_pivot["Type_Psychic-Flying"],new_pivot["Type_Psychic-Ghost"],new_pivot["Type_Psychic-Grass"],new_pivot["Type_Rock"],new_pivot["Type_Rock-Bug"],new_pivot["Type_Rock-Dark"],new_pivot["Type_Rock-Dragon"],new_pivot["Type_Rock-Fairy"],new_pivot["Type_Rock-Fighting"],new_pivot["Type_Rock-Flying"],new_pivot["Type_Rock-Grass"],new_pivot["Type_Rock-Ground"],new_pivot["Type_Rock-Ice"],new_pivot["Type_Rock-Psychic"],new_pivot["Type_Rock-Steel"],new_pivot["Type_Rock-Water"],new_pivot["Type_Steel"],new_pivot["Type_Steel-Dragon"],new_pivot["Type_Steel-Fairy"],new_pivot["Type_Steel-Fighting"],new_pivot["Type_Steel-Flying"],new_pivot["Type_Steel-Ghost"],new_pivot["Type_Steel-Ground"],new_pivot["Type_Steel-Psychic"],new_pivot["Type_Steel-Rock"],new_pivot["Type_Water"],new_pivot["Type_Water-Dark"],new_pivot["Type_Water-Dragon"],new_pivot["Type_Water-Electric"],new_pivot["Type_Water-Fairy"],new_pivot["Type_Water-Fighting"],new_pivot["Type_Water-Flying"],new_pivot["Type_Water-Ghost"],new_pivot["Type_Water-Grass"],new_pivot["Type_Water-Ground"],new_pivot["Type_Water-Ice"],new_pivot["Type_Water-Poison"],new_pivot["Type_Water-Psychic"],new_pivot["Type_Water-Rock"],new_pivot["Type_Water-Steel"])

/Users/miguel/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:3629: F_onewayBadInputSizesWarning: all input arrays have length 1.  f_oneway requires that at least one input has length greater than 1.
  warnings.warn(F_onewayBadInputSizesWarning(msg))


F_onewayResult(statistic=nan, pvalue=nan)

#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [67]:
a = pokemon[pokemon['unique_types']=='Grass-Poison']

In [70]:
a['Total'].sum()

6211

#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [5]:
# Your code here


#### Interpret the ANOVA test result. Is the difference significant?

In [6]:
# Your comment here
